In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('../train.csv')

In [25]:
def hygiene(
    X_train,
    *,
    max_nan_pct: float = 0.50,   # drop columns with >50% NaNs on TRAIN
    numeric_only: bool = True
):
    """
    Returns:
      keep_cols : list[str]  -> columns with NaN fraction <= max_nan_pct
      report    : dict       -> quick stats of what was dropped/kept
    """
    cols = X_train.select_dtypes(include=[np.number]).columns if numeric_only else X_train.columns
    na_pct = X_train[cols].isna().mean()           # fraction of NaNs per column
    keep_cols = na_pct[na_pct <= max_nan_pct].index.tolist()
    dropped = na_pct[na_pct > max_nan_pct].sort_values(ascending=False)

    if not keep_cols:  # safety: don't return empty schema
        keep_cols = list(cols)

    report = {
        "cols_considered": int(len(cols)),
        "kept_cols": int(len(keep_cols)),
        "dropped_cols": int(dropped.size),
        "max_nan_pct": float(max_nan_pct),
        "dropped_top5": dropped.head(5).to_dict()
    }
    return keep_cols, report


In [26]:
df['lagged_market_forward_excess_returns'] = df['market_forward_excess_returns'].shift(1)


In [27]:
df['lagged_risk_free_rate'] = df['risk_free_rate'].shift(1)

In [28]:
X = df.drop(columns = ['date_id', 'market_forward_excess_returns', 'risk_free_rate', 'forward_returns'])
y = df[['forward_returns', 'risk_free_rate', 'market_forward_excess_returns']]

In [29]:
keep_cols, report = hygiene(X, max_nan_pct = 0.2)

In [30]:
report

{'cols_considered': 96,
 'kept_cols': 83,
 'dropped_cols': 13,
 'max_nan_pct': 0.2,
 'dropped_top5': {'E7': 0.775194660734149,
  'V10': 0.6728587319243604,
  'S3': 0.6377085650723026,
  'M1': 0.6170189098998887,
  'M13': 0.6162402669632926}}

In [31]:
# 3) Lag features (time-safe)
def make_lag_features(df, cols, lags=(1, 5, 20), *, keep_original=True, dtype="float32"):
    """
    Returns a new DataFrame with optional originals + lagged copies.
    Uses shift(L), so expect NaNs at the head.
    """
    out = pd.DataFrame(index=df.index)
    if keep_original:
        out[cols] = df[cols]
    for L in lags:
        out[[f"{c}_lag{L}" for c in cols]] = df[cols].shift(L)
    if dtype is not None:
        for c in out.columns:
            if pd.api.types.is_float_dtype(out[c]):
                out[c] = out[c].astype(dtype)
    return out


In [32]:
# 4) Rolling stats (+ optional z-scores), time-safe via past_only=True
def make_rolling_features(
    df, cols,
    windows=(5, 20),
    *,
    stats=("mean", "std"),     # any of: "mean","std","min","max","sum"
    zscore: bool = False,       # z_t = (x_t - mean_{past}) / (std_{past}+eps)
    past_only: bool = True,    # shift(1) inside rolling to avoid leakage
    min_periods: int = None,  # default = window size
    eps: float = 1e-9,
    dtype: str = "float32",
):
    """
    Builds rolling features for each col over each window.
    If past_only=True, the rolling window excludes the current row (safe for t+1 targets).
    """
    out = pd.DataFrame(index=df.index)
    for w in windows:
        mp = w if min_periods is None else min_periods
        base = df[cols].shift(1) if past_only else df[cols]
        roll = base.rolling(window=w, min_periods=mp)

        if "mean" in stats:
            m = roll.mean()
            out[[f"{c}_rollmean{w}" for c in cols]] = m
        if "std" in stats:
            s = roll.std(ddof=0)
            out[[f"{c}_rollstd{w}" for c in cols]] = s
        if "min" in stats:
            out[[f"{c}_rollmin{w}" for c in cols]] = roll.min()
        if "max" in stats:
            out[[f"{c}_rollmax{w}" for c in cols]] = roll.max()
        if "sum" in stats:
            out[[f"{c}_rollsum{w}" for c in cols]] = roll.sum()

        if zscore:
            # need mean & std; compute if not already available
            if "mean" in stats:
                m = out[[f"{c}_rollmean{w}" for c in cols]].copy()
                m.columns = cols
            else:
                m = roll.mean()
            if "std" in stats:
                s = out[[f"{c}_rollstd{w}" for c in cols]].copy()
                s.columns = cols
            else:
                s = roll.std(ddof=0)
            for c in cols:
                out[f"{c}_z{w}"] = (df[c] - m[c]) / (s[c] + eps)

    if dtype is not None:
        for c in out.columns:
            if pd.api.types.is_float_dtype(out[c]):
                out[c] = out[c].astype(dtype)
    return out


In [33]:
X_enriched = make_rolling_features(X, keep_cols, windows = (5, 20, 60), stats = ("mean", "std"), zscore = True)

/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[[f"{c}_rollstd{w}" for c in cols]] = s
/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[[f"{c}_rollstd{w}" for c in cols]] = s
/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

In [34]:
X_enriched_cleaned = X_enriched.ffill().dropna(inplace = False)

In [35]:
X_enriched_cleaned

,D1_rollmean5,D2_rollmean5,D3_rollmean5,D4_rollmean5,D5_rollmean5,D6_rollmean5,D7_rollmean5,D8_rollmean5,D9_rollmean5,E1_rollmean5,...,V13_z60,V2_z60,V3_z60,V4_z60,V5_z60,V6_z60,V7_z60,V8_z60,lagged_market_forward_excess_returns_z60,lagged_risk_free_rate_z60
1844,0.0,0.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,2.185766,...,-1.411939,-0.972556,-0.298533,-0.199988,-0.457654,0.000000,-1.160732,0.000000,-0.350950,0.970820
1845,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.181245,...,-1.174448,0.717809,0.591987,0.310173,0.601800,0.000000,-1.162705,0.000000,0.465635,0.815838
1846,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.176746,...,-0.106902,0.245912,0.971053,0.059298,0.583099,0.000000,0.065339,0.000000,-0.549076,0.789853
1847,0.0,0.0,0.0,1.0,0.0,-0.2,0.0,0.0,0.0,2.172271,...,-0.515806,0.089773,-1.109595,-0.871174,1.383436,0.000000,-0.538323,0.000000,1.996067,0.888372
1848,0.0,0.0,0.0,1.0,0.0,-0.4,0.0,0.0,0.0,2.167817,...,0.279067,-0.521803,-0.566009,-0.867286,2.693936,0.000000,0.405928,0.000000,0.058054,1.478562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,0.0,0.0,0.0,0.0,0.8,0.0,0.2,0.4,0.6,1.572728,...,-0.839290,1.229938,0.231928,1.604594,0.857627,1.334903,-0.845946,1.843079,-0.697269,-2.788286
8986,0.0,0.0,0.0,0.0,0.6,0.0,0.2,0.2,0.6,1.570273,...,-0.828431,1.144157,0.938802,1.564443,-0.291685,1.195187,-0.863042,2.781567,0.199074,-3.855946
8987,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0,0.6,1.567826,...,-0.804145,1.004930,0.246196,1.355544,-0.188653,1.268582,-0.876705,1.453351,0.161481,-3.178122
8988,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.4,1.565386,...,-0.568384,0.852341,0.855848,1.319450,0.206112,1.403594,-0.663691,0.832501,0.261068,-2.720459


In [36]:
y = y.iloc[X_enriched_cleaned.index]

In [37]:
X_test = X_enriched_cleaned[-1000:]
X_val = X_enriched_cleaned[-2000:-1000]
X_train = X_enriched_cleaned[:-2000]
y_test = y[-1000:]
y_val = y[-2000:-1000]
y_train = y[:-2000]

In [38]:
X_val

,D1_rollmean5,D2_rollmean5,D3_rollmean5,D4_rollmean5,D5_rollmean5,D6_rollmean5,D7_rollmean5,D8_rollmean5,D9_rollmean5,E1_rollmean5,...,V13_z60,V2_z60,V3_z60,V4_z60,V5_z60,V6_z60,V7_z60,V8_z60,lagged_market_forward_excess_returns_z60,lagged_risk_free_rate_z60
6990,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.663594,...,-0.718964,0.760948,0.593287,0.713412,-0.513155,-0.264339,-0.968858,1.080518,0.736308,0.454673
6991,0.0,0.0,0.0,0.0,0.0,-0.8,0.0,0.2,0.0,0.662568,...,-0.919517,0.718234,0.666469,0.669194,-1.513536,-0.206116,-1.204478,1.045414,0.127694,0.308413
6992,0.0,0.0,0.0,0.0,0.2,-0.6,0.0,0.4,0.0,0.661542,...,-0.923600,0.707579,0.525649,0.542739,-0.813849,-0.085268,-1.215674,1.076759,0.645975,0.089529
6993,0.0,0.0,0.0,0.0,0.4,-0.4,0.0,0.6,0.0,0.660517,...,-0.964446,0.649531,0.651329,0.530414,0.164854,-0.823583,-1.245541,1.012096,0.812391,0.034375
6994,0.0,0.0,0.0,0.0,0.6,-0.2,0.0,0.6,0.2,0.659492,...,-0.913449,0.670378,0.646824,0.609735,0.189576,-0.807311,-1.181134,1.040495,0.316569,-0.356735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7985,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.4,1.608314,...,2.624379,1.634162,0.698517,2.010182,-1.427859,0.247978,2.290047,1.892456,-1.486409,-0.559135
7986,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,1.605683,...,1.226190,1.511876,0.283117,1.926646,-1.162139,0.416981,1.202320,1.785066,0.263555,-0.669419
7987,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.603060,...,1.194332,1.389627,-0.074592,1.844818,-1.641298,0.280935,1.228596,1.651045,-1.063217,-0.730449
7988,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.600445,...,0.302263,1.258981,-0.458043,1.775760,-1.934598,-0.727312,0.473459,1.318407,1.243035,-1.362838


In [39]:
X_test_raw = X.iloc[-(len(y_test)+65):]

In [40]:
X_train.to_csv('X_train.csv', index=False)
X_val.to_csv('X_val.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_val.to_csv('y_val.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
X_test_raw.to_csv('X_test_raw.csv', index=False)

In [41]:
X_test_raw

,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V2,V3,V4,V5,V6,V7,V8,V9,lagged_market_forward_excess_returns,lagged_risk_free_rate
7925,0,0,0,0,0,0,0,0,0,1.590116,...,0.445767,0.405423,0.339947,3.392064,0.330026,0.027822,0.472553,0.203687,-0.005891,7.936508e-07
7926,0,0,0,0,0,0,0,0,0,1.587533,...,0.417328,0.543651,0.338624,2.787521,0.417989,-0.020500,0.465939,0.164797,-0.000640,1.111111e-06
7927,0,0,0,0,0,0,0,0,0,1.584958,...,0.395503,0.487434,0.385582,3.331072,0.258598,0.333679,0.438823,0.432115,-0.013799,1.111111e-06
7928,0,0,0,0,0,-1,0,0,0,1.582391,...,0.404101,0.096561,0.392857,2.198840,0.544312,-0.004536,0.469246,0.182929,0.014007,1.111111e-06
7929,0,0,0,0,0,-1,0,0,0,1.579831,...,0.375000,0.624339,0.402778,3.022879,0.576720,-0.152657,0.412037,0.038272,0.005038,1.190476e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,0,0,0,0,0,0,0,0,0,1.565379,...,0.785053,0.469577,0.837963,1.226772,0.822751,-0.707361,0.142857,-0.649616,-0.003365,1.586508e-04
8986,0,0,0,0,0,0,0,0,0,1.562946,...,0.767857,0.671958,0.837963,0.785877,0.805556,-0.715692,0.196098,-0.668289,0.001990,1.551587e-04
8987,0,0,1,0,0,0,0,0,0,1.560520,...,0.734127,0.481481,0.787698,0.834898,0.823413,-0.723949,0.133929,-0.670946,0.001845,1.558730e-04
8988,0,0,0,0,0,0,0,0,0,1.558102,...,0.695106,0.655423,0.783730,0.994026,0.851852,-0.684937,0.101852,-0.646265,0.002424,1.564683e-04


In [42]:
list(set(X_test.columns) - set(make_rolling_features(X_test_raw, X_test_raw.columns, windows = (5, 20, 60), stats = ['means', 'std'], zscore = True).columns))

/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_z{w}"] = (df[c] - m[c]) / (s[c] + eps)
/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_z{w}"] = (df[c] - m[c]) / (s[c] + eps)
/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

['E9_rollmean60',
 'E1_rollmean60',
 'V7_rollmean20',
 'P2_rollmean60',
 'E19_rollmean5',
 'M12_rollmean60',
 'E11_rollmean60',
 'E9_rollmean5',
 'D5_rollmean60',
 'E2_rollmean60',
 'E6_rollmean20',
 'S4_rollmean5',
 'I1_rollmean5',
 'M17_rollmean5',
 'D3_rollmean60',
 'M8_rollmean20',
 'E2_rollmean20',
 'V7_rollmean60',
 'lagged_risk_free_rate_rollmean5',
 'E16_rollmean5',
 'V11_rollmean5',
 'E9_rollmean20',
 'D2_rollmean5',
 'I5_rollmean60',
 'P5_rollmean5',
 'E4_rollmean5',
 'E8_rollmean20',
 'M16_rollmean60',
 'M12_rollmean5',
 'V4_rollmean5',
 'V12_rollmean5',
 'P2_rollmean5',
 'P7_rollmean60',
 'S11_rollmean20',
 'E11_rollmean20',
 'I2_rollmean5',
 'M9_rollmean20',
 'P8_rollmean5',
 'D1_rollmean5',
 'M16_rollmean5',
 'P11_rollmean60',
 'I4_rollmean20',
 'S7_rollmean20',
 'I7_rollmean60',
 'P7_rollmean20',
 'D9_rollmean60',
 'E8_rollmean5',
 'D9_rollmean20',
 'M7_rollmean5',
 'E20_rollmean60',
 'E16_rollmean60',
 'M15_rollmean20',
 'I3_rollmean60',
 'E10_rollmean20',
 'M17_rollmea

In [43]:
X_test_raw.columns

Index(['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E10',
       'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E2',
       'E20', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'I1', 'I2', 'I3', 'I4',
       'I5', 'I6', 'I7', 'I8', 'I9', 'M1', 'M10', 'M11', 'M12', 'M13', 'M14',
       'M15', 'M16', 'M17', 'M18', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8',
       'M9', 'P1', 'P10', 'P11', 'P12', 'P13', 'P2', 'P3', 'P4', 'P5', 'P6',
       'P7', 'P8', 'P9', 'S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S4', 'S5',
       'S6', 'S7', 'S8', 'S9', 'V1', 'V10', 'V11', 'V12', 'V13', 'V2', 'V3',
       'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'lagged_market_forward_excess_returns', 'lagged_risk_free_rate'],
      dtype='object')

In [44]:
make_rolling_features(X_test_raw, X_test_raw.columns, windows = (5, 20, 60), stats = ['mean', 'std', 'min', 'max'], zscore = True)

/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[[f"{c}_rollstd{w}" for c in cols]] = s
/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[[f"{c}_rollstd{w}" for c in cols]] = s
/var/folders/l5/hb787m6n4z3b6q0_q9g7xpc80000gn/T/ipykernel_1734/807630681.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

,D1_rollmean5,D2_rollmean5,D3_rollmean5,D4_rollmean5,D5_rollmean5,D6_rollmean5,D7_rollmean5,D8_rollmean5,D9_rollmean5,E1_rollmean5,...,V2_z60,V3_z60,V4_z60,V5_z60,V6_z60,V7_z60,V8_z60,V9_z60,lagged_market_forward_excess_returns_z60,lagged_risk_free_rate_z60
7925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,0.0,0.0,0.0,0.0,0.8,0.0,0.2,0.4,0.6,1.572728,...,1.229938,0.231928,1.604594,0.857627,1.334903,-0.845946,1.843079,-0.886227,-0.697269,-2.788286
8986,0.0,0.0,0.0,0.0,0.6,0.0,0.2,0.2,0.6,1.570273,...,1.144157,0.938802,1.564443,-0.291685,1.195187,-0.863042,2.781567,-0.966815,0.199074,-3.855946
8987,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0,0.6,1.567826,...,1.004930,0.246196,1.355544,-0.188653,1.268582,-0.876705,1.453351,-0.950699,0.161481,-3.178122
8988,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.4,1.565386,...,0.852341,0.855848,1.319450,0.206112,1.403594,-0.663691,0.832501,-0.777634,0.261068,-2.720459
